In [2]:
import json

In [3]:

with open('../data/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

In [6]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['context']}-{doc['question']}-{doc['answer'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [7]:

for doc in documents:
    doc['id'] = generate_document_id(doc)

In [8]:
documents[3]

{'context': "Find Movers in Santa Clarita, CA 91390 - Moving Company 91390\nWhether you're looking for just the right movers in Santa Clarita, Help with your move or ways to cut the cost of moving. Pearl Moving Company Santa Clarita, 91390 is working hard to connect you with the best moving companies in Santa Clarita, We are here to make your move as easy as possible.\nWith our Moving companies In Santa Clarita Clarita.50 • 91380 • 91382 • 91383 • 91390 •\nApartment Movers 91390, Local Movers 91390, Movers 91390, Moving Company 91390, Moving Services 91390, Office Move 91390, home movers 91390, Out of State Movers 91390, Professional Movers 91390, Moving Companies 91390, Cheap Movers 91390, movers reviews 91390, movers cost 91390, home moving companies 91390, moving and storage companies 91390, Long Distance Moving Company 91390, moving company reviews 91390, commercial movers 91390,Moving Quotes 91390, local moving companies 91390, local moving company 91390, best moving companies 913

In [9]:
from collections import defaultdict

In [10]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [11]:
len(hashes), len(documents)

(960, 960)

In [12]:
with open('../data/documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [17]:
prompt_template = """
You emulate an interviewer who is doing the interview with an expert.
Formulate 5 questions this student might ask based on a  record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

context: {context}
question: {question}
answer: {answer}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [15]:
from groq import Groq

In [ ]:
client = Groq()

In [ ]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": "Is it too late to join the course",
        }
    ],
)

In [18]:

def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gemma2-9b-it',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [19]:

from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:

results = {}

In [21]:

for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  7%|█▍                   | 66/960 [03:30<47:31,  3.19s/it]


KeyboardInterrupt: 

In [22]:
results

{'add14665': '[\n"What prompted the search for information related to a \'collar george herbert essay\'?",\n"What type of resource yielded the answer to the search query?",\n"What other related topics, aside from the essay itself, were mentioned in conjunction with the search query?",\n"Based on the presence of other essay writing resources, what can be inferred about the individual\'s needs or intentions?",\n"What is the potential significance of a search query like \'collar george herbert essay\'?"\n] \n',
 '1e9a1431': '[\n"What are some instances in which charitable giving, as described by the author, might actually make a social situation worse?",\n"According to the author, what are some key ethical issues that can arise from a \'give-back\' business model?",\n"How does the author critique the argument that companies should prioritize profit and then give back to charity?",\n"What is the author\'s opinion on the role of business in addressing social issues, beyond simple charitable

In [23]:

parsed_resulst = {}

for doc_id, json_questions in results.items():
    if 
    parsed_resulst[doc_id] = json.loads(json_questions)

JSONDecodeError: Expecting ',' delimiter: line 1 column 57 (char 56)

In [24]:

doc_index = {d['id']: d for d in documents}

In [26]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    context = doc_index[doc_id]['context']
    for q in questions:
        final_results.append((q, context, doc_id))

In [27]:

import pandas as pd


In [28]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [29]:
df

,question,course,document
0,What prompted the search for information relat...,Francisco Rogers found the answer to a search ...,add14665
1,What type of resource yielded the answer to th...,Francisco Rogers found the answer to a search ...,add14665
2,"What other related topics, aside from the essa...",Francisco Rogers found the answer to a search ...,add14665
3,Based on the presence of other essay writing r...,Francisco Rogers found the answer to a search ...,add14665
4,What is the potential significance of a search...,Francisco Rogers found the answer to a search ...,add14665
...,...,...,...
118,What is the type of sensor used in the camera?,"Hey guys, I know I’ve been a little slow to po...",467590f9
119,What size is the LCD screen and what is its re...,"Hey guys, I know I’ve been a little slow to po...",467590f9
120,What is the camera's zoom range and its wide a...,"Hey guys, I know I’ve been a little slow to po...",467590f9
121,What is the camera's weight and does it allow ...,"Hey guys, I know I’ve been a little slow to po...",467590f9
